# Imports

##### General imports

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../")

In [2]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import copy

##### Import from flatland environment 

In [3]:
from flatland.envs.rail_env import RailEnv
from flatland.envs.observations import *
from flatland.envs.rail_generators import complex_rail_generator,rail_from_manual_specifications_generator,random_rail_generator, RailGenerator,sparse_rail_generator
from flatland.envs.schedule_generators import complex_schedule_generator, random_schedule_generator, ScheduleGenerator, sparse_schedule_generator
from flatland.utils.rendertools import RenderTool, AgentRenderVariant

##### Import from our framework

In [4]:
from src.graph import NetworkGraph
from src.flows import TimeNetwork
from src.flows import MCFlow

# Test of time expanded network

##### Create a flatland network

In [5]:
number_agents = 5

size_side = 50
stochastic_data = {'prop_malfunction': 0.3,  # Percentage of defective agents
                   'malfunction_rate': 30,  # Rate of malfunction occurence
                   'min_duration': 3,  # Minimal duration of malfunction
                   'max_duration': 20  # Max duration of malfunction
                   }
speed_ration_map = {1.: 0.25,  # Fast passenger train
                    1. / 2.: 0.25,  # Fast freight train
                    1. / 3.: 0.25,  # Slow commuter train
                    1. / 4.: 0.25}  # Slow freight train
env = RailEnv(width=size_side,
              height=size_side,
              rail_generator=sparse_rail_generator(max_num_cities=4,
                                                   # Number of cities in map (where train stations are)
                                                   seed=14,  # Random seed
                                                   grid_mode=False,
                                                   max_rails_between_cities=2,
                                                   max_rails_in_city=8,
                                                   ),
              schedule_generator=sparse_schedule_generator(speed_ration_map),
              number_of_agents=number_agents,
              stochastic_data=stochastic_data,  # Malfunction data generator
              obs_builder_object=GlobalObsForRailEnv(),
              remove_agents_at_target=True
              )

# RailEnv.DEPOT_POSITION = lambda agent, agent_handle : (agent_handle % env.height,0)
env.reset()

env_renderer = RenderTool(env,
                          agent_render_variant=AgentRenderVariant.AGENT_SHOWS_OPTIONS_AND_BOX,
                          show_debug=True,
                          screen_height=1100,
                          screen_width=1800)
env_renderer.reset()

env_renderer.render_env(show=True, show_observations=False, show_predictions=False)

matrix_rail = np.array(env.rail.grid.tolist())
flatlandNetwork = NetworkGraph(matrix_rail,[(0,1)],[(1,0)])

In [16]:
env.agents

[EnvAgent(initial_position=(26, 39), direction=0, target=(7, 39), moving=False, speed_data={'position_fraction': 0.0, 'speed': 0.5, 'transition_action_on_cellexit': 0}, malfunction_data={'malfunction': 0, 'malfunction_rate': 0.0, 'next_malfunction': 0, 'nr_malfunctions': 0}, status=<RailAgentStatus.READY_TO_DEPART: 0>, position=None, handle=0, old_direction=None, old_position=None),
 EnvAgent(initial_position=(7, 38), direction=2, target=(26, 44), moving=False, speed_data={'position_fraction': 0.0, 'speed': 0.3333333333333333, 'transition_action_on_cellexit': 0}, malfunction_data={'malfunction': 0, 'malfunction_rate': 0.0, 'next_malfunction': 0, 'nr_malfunctions': 0}, status=<RailAgentStatus.READY_TO_DEPART: 0>, position=None, handle=1, old_direction=None, old_position=None),
 EnvAgent(initial_position=(12, 9), direction=2, target=(7, 41), moving=False, speed_data={'position_fraction': 0.0, 'speed': 1.0, 'transition_action_on_cellexit': 0}, malfunction_data={'malfunction': 0, 'malfunct

In [6]:
sources = []
sinks = []
for agent in env.agents:
    sources.append(agent.initial_position)
    sinks.append(agent.target)

##### create a time expanded network

In [7]:
import time

In [8]:
start = time.time()
TestNetworkTime = TimeNetwork(flatlandNetwork, depth=80)
stop = time.time()
print(f'time taken to build the graph: {stop-start}')

time taken to build the graph: 3.014137029647827


In [9]:
TestNetworkTime.connect_sources_and_sink(sources,sinks)

##### Test LP Formulation

test a simple graph

In [10]:
import datetime as dt
print(dt.datetime.now())

2019-11-10 13:30:44.449784


In [11]:
mcflow = MCFlow(TestNetworkTime.graph,len(sources),TestNetworkTime.get_topology_network(),integer = False)

Academic license - for non-commercial use only


In [12]:
mcflow.solve()

Optimize a model with 644110 rows, 1185650 columns and 3843300 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 587016 rows and 1176987 columns
Presolve time: 1.26s

Solved with barrier
Solved in 0 iterations and 1.31 seconds
Infeasible model
model not optimized or failed to optimize 
 please use .solve() and check the output
no collision detected


In [13]:
paths = mcflow.get_paths_solution()

In [14]:
lengths = []
pathsToAllongate = copy.deepcopy(mcflow.solution)
for agent,path in pathsToAllongate.items():
    lengths.append(len(path))
    
maxLength = max(lengths)
for agent,path in pathsToAllongate.items():
    for i in range(maxLength-len(path)):
        path.append(None)
    pathsToAllongate[agent] = path

ValueError: max() arg is an empty sequence

In [ ]:
dfPaths = pd.DataFrame(pathsToAllongate)
dfPaths

In [ ]:
mcflow.show_results()

In [ ]:
import datetime as dt
print(dt.datetime.now())